In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
folder_path = "/kaggle/input/fashion-product-images-small/images"
file_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
print(len(file_names))

44441


In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv("/kaggle/input/fashion-product-images-small/styles.csv", on_bad_lines="skip")
current = []
for x in df.id.values:
    if str(x)+".jpg" not in file_names:
        current.append(False)
    else:
        current.append(True)
df["Good"] = np.array(current)
df.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,Good
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,True
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,True
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch,True
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants,True
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt,True


In [5]:
df["baseColour"].unique()

array(['Navy Blue', 'Blue', 'Silver', 'Black', 'Grey', 'Green', 'Purple',
       'White', 'Beige', 'Brown', 'Bronze', 'Teal', 'Copper', 'Pink',
       'Off White', 'Maroon', 'Red', 'Khaki', 'Orange', 'Coffee Brown',
       'Yellow', 'Charcoal', 'Gold', 'Steel', 'Tan', 'Multi', 'Magenta',
       'Lavender', 'Sea Green', 'Cream', 'Peach', 'Olive', 'Skin',
       'Burgundy', 'Grey Melange', 'Rust', 'Rose', 'Lime Green', 'Mauve',
       'Turquoise Blue', 'Metallic', 'Mustard', 'Taupe', 'Nude',
       'Mushroom Brown', nan, 'Fluorescent Green'], dtype=object)

In [3]:
idx = []
category = []
color = []
for a, b, c, d in zip(df.id.values, df.masterCategory.values, df.baseColour.values, df.Good.values):
    if d:
        idx.append(a)
        category.append(b)
        color.append(c)

In [8]:
data = pd.DataFrame()
data["id"] = np.array(idx)
data["masterCategory"] = np.array(category)
data["baseColour"] = np.array(color)

In [9]:
colors = ['Black', 'White', 'Blue', 'Brown', 'Grey', 'Red', 'Green', 'Pink', 'Navy Blue', 'Purple']
preprocess = {}
for x, y in enumerate(colors):
    preprocess[y] = x
for x in list(data["baseColour"].unique()):
    if x not in colors:
        preprocess[x] = len(colors)

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim 
import torchvision.transforms as transforms
import torchvision
import os
import pandas as pd
from PIL import Image
from torch.utils.data import (
    Dataset,
    DataLoader,
)

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to the same size
    transforms.ToTensor(),
])

class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = data
#         self.preprocess = {}
#         for x, y in enumerate(self.annotations["baseColour"].unique()):
#             self.preprocess[y] = x
        self.annotations["baseColour"] = self.annotations["baseColour"].map(preprocess)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, str(self.annotations.iloc[index, 0])+".jpg")
        image = Image.open(img_path).convert("RGB")
        y_label = torch.tensor(int(self.annotations.iloc[index, 2]))
        if self.transform:
            image = self.transform(image)
        return (image, y_label)
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
in_channel = 3
num_classes = 11
learning_rate = 3e-4
batch_size = 64
num_epochs = 5

# Load Data
dataset = CustomDataset(
    csv_file="/kaggle/input/fashion-product-images-small/styles.csv",
    root_dir="/kaggle/input/fashion-product-images-small/images",
    transform=transform,
)

train_set, test_set = torch.utils.data.random_split(dataset, [35000, 9419])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

# Model

model = torchvision.models.googlenet(pretrained = True)

for param in model.parameters():
    param.requires_grad = True

# final layer is not frozen
flts = model.fc.in_features
model.fc = nn.Linear(in_features=flts, out_features=num_classes)
model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth
100%|██████████| 49.7M/49.7M [00:03<00:00, 13.8MB/s]


In [11]:
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")

# Check accuracy on training to see how good our model is
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}")

    model.train()

print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)

print("Checking accuracy on Test Set")
check_accuracy(test_loader, model)

Cost at epoch 0 is 1.0503487552105835
Cost at epoch 1 is 0.8417118524066709
Cost at epoch 2 is 0.7438353151355409
Cost at epoch 3 is 0.653000813461091
Cost at epoch 4 is 0.5679955850244658
Checking accuracy on Training Set
Got 29498 / 35000 with accuracy 84.28
Checking accuracy on Test Set
Got 6711 / 9419 with accuracy 71.25


In [12]:
torch.save(model, 'fashion-color-googlenet-10c.pth')

In [13]:
torch.save(model.state_dict(), "fashion-color-googlenet-10c-state-dict.pth")